In [0]:
#Set google drive
from google.colab import auth
auth.authenticate_user()

from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
#Install miniconda and rdkit
!wget -c https://repo.continuum.io/miniconda/Miniconda3-latest-Linux-x86_64.sh
!chmod +x Miniconda3-latest-Linux-x86_64.sh
!time bash ./Miniconda3-latest-Linux-x86_64.sh -b -f -p /usr/local
!time conda install -q -y -c conda-forge rdkit
import sys
sys.path.append('/usr/local/lib/python3.7/site-packages/')

--2019-11-20 00:32:55--  https://repo.continuum.io/miniconda/Miniconda3-latest-Linux-x86_64.sh
Resolving repo.continuum.io (repo.continuum.io)... 104.18.200.79, 104.18.201.79, 2606:4700::6812:c84f, ...
Connecting to repo.continuum.io (repo.continuum.io)|104.18.200.79|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 71785000 (68M) [application/x-sh]
Saving to: ‘Miniconda3-latest-Linux-x86_64.sh’

Miniconda3-latest-L 100%[===================>]  68.46M   153MB/s    in 0.4s    

2019-11-20 00:32:55 (153 MB/s) - ‘Miniconda3-latest-Linux-x86_64.sh’ saved [71785000/71785000]

PREFIX=/usr/local
Unpacking payload ...
Solving environment: / - done

## Package Plan ##

  environment location: /usr/local

  added / updated specs:
    - _libgcc_mutex==0.1=main
    - asn1crypto==1.2.0=py37_0
    - ca-certificates==2019.10.16=0
    - certifi==2019.9.11=py37_0
    - cffi==1.13.0=py37h2e261b9_0
    - chardet==3.0.4=py37_1003
    - conda-package-handling==1.6.0=py37h7b6447c_

In [0]:
#Read smiles files
from rdkit import Chem
from rdkit.Chem.Crippen import MolLogP
import numpy as np
import torch
import time

maxlen = 64
with open('/content/gdrive/My Drive/Colab Notebooks/smiles.txt') as f:
  smiles = f.readlines()[:]
  smiles = [s.strip() for s in smiles]
  smiles = [s.split()[1] for s in smiles]
  smiles = [s for s in smiles if len(s)<maxlen]

#Characters of smiles
all_smiles=''
for s in smiles: all_smiles+=s
chars = sorted(list(set(list(all_smiles))))
chars.append('X')
c_to_i = {c:i for i,c in enumerate(chars)}
i_to_c = {i:c for i,c in enumerate(chars)}
print ('Max len:', maxlen)
print ('Number of chars:', len(chars))
print (len(smiles))
print (c_to_i)

Max len: 64
Number of chars: 46
448984
{'#': 0, '(': 1, ')': 2, '+': 3, '-': 4, '.': 5, '/': 6, '1': 7, '2': 8, '3': 9, '4': 10, '5': 11, '6': 12, '7': 13, '=': 14, '@': 15, 'B': 16, 'C': 17, 'F': 18, 'G': 19, 'H': 20, 'I': 21, 'K': 22, 'L': 23, 'M': 24, 'N': 25, 'O': 26, 'P': 27, 'S': 28, 'V': 29, 'Z': 30, '[': 31, '\\': 32, ']': 33, 'a': 34, 'c': 35, 'e': 36, 'g': 37, 'i': 38, 'l': 39, 'n': 40, 'o': 41, 'r': 42, 's': 43, 'u': 44, 'X': 45}


In [0]:
#Dataset
from torch.utils.data import Dataset, DataLoader

class MolDataset(Dataset):
  def __init__(self, smiles, c_to_i):
    self.c_to_i = c_to_i
    self.smiles = smiles
    
  def __len__(self):
      return len(self.smiles)

  def __getitem__(self, idx):
    s = self.smiles[idx] + 'X'
    i = torch.from_numpy(np.array([c_to_i[c] for c in s]))
    sample = dict()
    sample['X'] = i
    sample['L'] = len(s) - 1
    return sample

In [0]:
#collate function
#Collate fn
def my_collate(batch):
    sample = dict()
    X =  torch.nn.utils.rnn.pad_sequence([b['X'] for b in batch], batch_first=True, padding_value = 45)
    L = torch.Tensor([b['L'] for b in batch])

    sample['X'] = X
    sample['L'] = L

    return sample

In [0]:
#model
#feature forcing
#rnn

#mu, variance
#randn_like(std)
#torch.nn.CrossEntropyLoss
#start codon - change to learnable parameter
#decoder - F.softmax

import torch
import torch.nn as nn
import torch.nn.functional as F

class VAE(torch.nn.Module):
  def __init__(self, n_feature=128, n_rnn_layer = 3, dropout=0.1):
    super(VAE, self).__init__()
    self.encoder = nn.GRU(input_size=n_feature, hidden_size=n_feature, num_layers=n_rnn_layer)
    self.decoder = nn.GRU(input_size=n_feature*2, hidden_size=n_feature, num_layers=n_rnn_layer)

    self.mu = nn.Linear(n_feature, n_feature)
    self.logvar = nn.Linear(n_feature, n_feature)
    
    self.fc = nn.Linear(n_feature, 46)
    self.embedding = nn.Embedding(46, n_feature)
    self.n_feature = n_feature
    self.n_rnn_layer = n_rnn_layer
    self.start = torch.nn.Parameter(torch.randn(1, 1, n_feature), requires_grad = True)
    #self.stcodon.requires_grad = True

  def reparameterize(self, mu, logvar):
      std = torch.exp(0.5*logvar)
      eps = torch.randn_like(std)
      return mu + eps*std

  def forward(self, x, l):
    x = self.embedding(x)
    x = x.permute((1,0,2)) # batch, sequence, channel
    
    enc_out, h_n = self.encoder(x)
    selected = []

    for i in range(len(l)):
      selected.append(enc_out[l[i]-1, i])

    selected = torch.stack(selected)

    mu = self.mu(selected)
    logvar = self.logvar(selected)

    z = self.reparameterize(mu, logvar)
    z = z.unsqueeze(0)

    dec_in = torch.cat((self.start.repeat(1, x.size(1), 1), x), 0)
    repeat_z = z.repeat(dec_in.size(0),1 ,1)
    dec_in = torch.cat((dec_in, repeat_z) , dim=-1)
    dec_out, h_n = self.decoder(dec_in)
    retval = self.fc(dec_out[:-1])
    retval = retval.permute((1,0,2))

    return retval, mu, logvar

  def sample(self):
    #sampling z
    z = torch.randn((1, self.n_feature)).cuda()
    z = z.unsqueeze(0)

    dec_in = self.start
    result = []
    hidden = torch.zeros((self.n_rnn_layer, 1, self.n_feature)).cuda()

    for i in range(120):
      dec_in = torch.cat((dec_in, z), dim=-1)
      dec_out, h_n = self.decoder(dec_in, hidden)
      hidden = h_n
      retval = torch.distributions.categorical.Categorical(F.softmax(self.fc(dec_out), dim=-1)).sample()
      result.append(retval.item())
      dec_in = self.embedding(retval)

    return result




In [0]:
#Train model
def loss_function(recon_x, x, mu, logvar, l):
  loss_fn = torch.nn.CrossEntropyLoss(reduction='sum')
  CEL = 0
  for i in range(len(l)):
    CEL += loss_fn(recon_x[i, :l[i]+1], x[i, :l[i]+1])
  KLD = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())

  return CEL + KLD

lr = 1e-4

model = VAE(512).cuda()
model_list  = []
result = []


#Dataset
train_smiles = smiles[:100000]
train_dataset = MolDataset(train_smiles, c_to_i)

#Dataloader
train_dataloader = DataLoader(train_dataset, batch_size=128, num_workers=0, collate_fn=my_collate)

optimizer = torch.optim.Adam(model.parameters(), lr=lr)
#optimizer = torch.optim.SGD(model.parameters(), lr=lr)


fn = '/content/gdrive/My Drive/Models/save'+ str(21) +'.pt'
model.load_state_dict(torch.load(fn))

for epoch in range(100):
  for i_batch, batch in enumerate(train_dataloader):
    optimizer.zero_grad()
    x, l = \
      batch['X'].cuda().long(), batch['L'].cuda().long()
    out_x, mu, logvar = model(x, l)
    loss = loss_function(out_x, x, mu, logvar, l)
    loss.backward()
    torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
    optimizer.step()

  if True: 
    fn = '/content/gdrive/My Drive/Models/save'+ str(epoch+22) +'.pt'
    torch.save(model.state_dict(), fn)
    generated = model.sample()
    generated = ''.join([i_to_c[i] for i in generated])
    generated = generated.split('X')[0]
    print(epoch, loss)
    print(generated)
    m = Chem.MolFromSmiles(generated)
    if m is None:
      print('invalid')




KeyboardInterrupt: ignored

In [0]:
#Test model
model = VAE(512).cuda()

for epoch in [1, 2, 5, 10, 20, 40, 50, 60, 80, 100]:
  fn = '/content/gdrive/My Drive/Models/save'+ str(epoch) +'.pt'
  model.load_state_dict(torch.load(fn))
  model.eval()
  valid = 0
  for i in range(100):
    generated = model.sample()
    generated = ''.join([i_to_c[i] for i in generated])
    generated = generated.split('X')[0]
    m = Chem.MolFromSmiles(generated)
    if m is None:
      continue
    valid += 1
    
  print(epoch, valid, generated) 
  if m is None:
    print('invalid')

1 27 CCOc1ccc(Nc2nc(-c3ccccc3)cc3c(c2c2O)CC3c2ccc(Br)c(OC)c2OC3
invalid
2 39 CCOc1cc(OC)ccc1NC(=O)c1c(NC(=O)c2ccc(Cl)cc2)[nH]c2ccccc12)CCCCC2
invalid
5 52 O=c1nc(Nc2nc(Sc3nc4ccccc4n(Cc4ccccc4F)c3n2)CC(C)(N)Oc3ccccc32)ccc1OCCC(C)C
invalid
10 72 COc1cccc(/C=N/Nc2nc(-c3ccccc3)nc3ccccc23)c1
20 83 CCOC(=O)c1c(-c2ccccc2)oc2ccc(OC(=O)c3ccc(C)cc3)cc12
40 90 CCc1nc2c(c(=O)n1C1CCCCC1)c1nc3ccccc3nc1n2C1CCCCC1
50 90 O=C(CSc1ccc(Cl)cc1)Nc1ccc2c(c1)CCCC2
60 84 COc1cccc(/C=N/NC(=O)CSc2ccccn2)c1O
80 87 CCOC(=O)c1c(-c2ccc(OC)cc2)oc2ccc(OCc3ccccc3Br)cc12
100 89 CN(C)c1ccc(C(c2c(O)ccc3ccc(C(=O)O)cc3)o2)cc1
invalid
